# Day 15: Chiton

In [1]:
example = """1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581"""

In [33]:
def parse(input):
    """Parses weights from input."""
    return [[int(char) for char in line] for line in input.splitlines()]
    
parse(example)

[[1, 1, 6, 3, 7, 5, 1, 7, 4, 2],
 [1, 3, 8, 1, 3, 7, 3, 6, 7, 2],
 [2, 1, 3, 6, 5, 1, 1, 3, 2, 8],
 [3, 6, 9, 4, 9, 3, 1, 5, 6, 9],
 [7, 4, 6, 3, 4, 1, 7, 1, 1, 1],
 [1, 3, 1, 9, 1, 2, 8, 1, 3, 7],
 [1, 3, 5, 9, 9, 1, 2, 4, 2, 1],
 [3, 1, 2, 5, 4, 2, 1, 6, 3, 9],
 [1, 2, 9, 3, 1, 3, 8, 5, 2, 1],
 [2, 3, 1, 1, 9, 4, 4, 5, 8, 1]]

Distance of shortest path in example should be 40.

In [28]:
import math

def find_path(origin, dest, weights):
    """Finds shortest path to dest using Dijkstra's algorithm."""
    # 1. Mark all nodes unvisited. 
    unvisited = {(x, y) for x in range(len(weights)) for y in range(len(weights[0]))}
    visited = set()
    
    # 2. Assign tentative distances to all nodes: 0 for origin node, and infinity for all others.
    distances = {node: math.inf for node in unvisited}
    distances[origin] = 0
    
    x, y = origin
    while unvisited:
        # 3. For the current node, consider all of its unvisited neighbours.
        for i, j in [node for node in ((x - 1, y), (x, y + 1), (x + 1, y), (x, y - 1)) if node in unvisited]:
            # Assign the smaller of new tentative distance or currently assigned distance.
            if distances[i, j] > distances[x, y] + weights[i][j]:
                distances[i, j] = distances[x, y] + weights[i][j]
        
        # 4. Mark the current node as visited.
        unvisited.remove((x, y))
        visited.add((x, y))
                     
        # 5. If the destination has been marked visited, stop!
        if dest in visited:
            break
            
        # 6. Set current node to unvisited node with smallest tentative distance.
        x, y = min(unvisited, key=lambda node: distances[node])
    return distances

find_path((0, 0), (9, 9), parse(example))[9, 9]

40

Find distance of shortest path in input.

In [37]:
weights = parse(open('day-15-input.txt').read())
find_path((0, 0), (len(weights) - 1, len(weights[0]) - 1), weights)[len(weights) - 1, len(weights[0]) - 1]

390

# Part two

I only got a result after running this for a very long time. This solution is not good enough.

The full 5x5 map can be calculated from the original tile.

In [60]:
parse(example)

[[1, 1, 6, 3, 7, 5, 1, 7, 4, 2],
 [1, 3, 8, 1, 3, 7, 3, 6, 7, 2],
 [2, 1, 3, 6, 5, 1, 1, 3, 2, 8],
 [3, 6, 9, 4, 9, 3, 1, 5, 6, 9],
 [7, 4, 6, 3, 4, 1, 7, 1, 1, 1],
 [1, 3, 1, 9, 1, 2, 8, 1, 3, 7],
 [1, 3, 5, 9, 9, 1, 2, 4, 2, 1],
 [3, 1, 2, 5, 4, 2, 1, 6, 3, 9],
 [1, 2, 9, 3, 1, 3, 8, 5, 2, 1],
 [2, 3, 1, 1, 9, 4, 4, 5, 8, 1]]

The top-right element should be 6.

In [125]:
def transform(x, y, tile):
    """Returns value of (x, y) in full 5x5 map."""
    transformed = tile[x % len(tile)][y % len(tile[0])] + (x // len(tile)) + (y // len(tile[0]))
    return transformed if transformed < 10 else transformed % 9

transform(0, 49, parse(example))

6

Bottom-right of original tile shoule be unchanged - 1.

In [104]:
transform(9, 9, parse(example))

1

Diagonals should be incremented twice - 3 in this case.

In [105]:
transform(10, 10, parse(example))

3

High values should wrap (9 -> 11 -> 2)

In [127]:
transform(15, 13, parse(example))

2

Shortest past in 5x5 example should be 315.

In [129]:
import collections

def find_path2(origin, dest, weights):
    """Finds shortest path to dest on full 5x5 map using Dijkstra's algorithm.
    
    Store distances sparsely this time.
    """
    # 1. Mark all nodes unvisited.
    # There are 25x more nodes this time.
    unvisited = {(x, y) for x in range(len(weights) * 5) for y in range(len(weights[0]) * 5)}
    visited = set()
    
    # 2. Assign tentative distances to all nodes: 0 for origin node, and infinity for all others.
    distances = collections.defaultdict(lambda: math.inf)
    distances[origin] = 0
    
    x, y = origin
    while unvisited:
        # 3. For the current node, consider all of its unvisited neighbours.
        for i, j in [node for node in ((x - 1, y), (x, y + 1), (x + 1, y), (x, y - 1)) if node in unvisited]:
            # Assign the smaller of new tentative distance or currently assigned distance.
            if distances[i, j] > distances[x, y] + transform(i, j, weights):
                distances[i, j] = distances[x, y] + transform(i, j, weights)
        
        # 4. Mark the current node as visited.
        unvisited.remove((x, y))
        visited.add((x, y))
                     
        # 5. If the destination has been marked visited, stop!
        if dest in visited:
            break
            
        # 6. Set current node to unvisited node with smallest tentative distance.
        x, y = min(unvisited, key=lambda node: distances[node])
    return distances

weights = parse(example)
dest = (len(weights) * 5 - 1, len(weights[0]) * 5 - 1)
find_path2((0, 0), dest, weights)[dest[0], dest[1]]

315

Find distance of shortest path in 5x5 transformed input.

In [130]:
weights = parse(open('day-15-input.txt').read())
dest = (len(weights) * 5 - 1, len(weights[0]) * 5 - 1)
find_path2((0, 0), dest, weights)[dest[0], dest[1]]

2814